# 02. Procesarea Datelor

In [15]:
! pip install pyodbc
! pip install sqlalchemy

In [16]:
import pandas as pd
import json
import pyodbc
import sqlalchemy as db
import urllib

exec(open('azure_credentials.py').read())

In [17]:
username

'aid2023adminlog'

### Load tabele

In [18]:
# Load csv school_details
details_Team4 = pd.read_csv('school_details.csv')
details_Team4.head(3)

,idSchool,internalIdSchool,idParentSchool,schoolYearDescription,code,siruesCode,longName,shortName,schoolType,statut,...,streetNumber,postalCode,phoneNumber,faxNumber,email,idSchoolYear.code,schoolNumbers.idSchool,schoolNumbers.studyFormationsCount,schoolNumbers.studentsCount,schoolNumbers.personnelCount
0,11269910,10131270,NaN,Anul şcolar 2018-2019,261200154,23054595,"ASOCIAŢIA CREŞTINĂ DE CARITATE ""GOSEN"" ARAD/ G...","GRĂDINIŢA ""GOSEN"" ARAD",Unitate de învățământ,Cu personalitate juridică,...,121-125,310365.0,257272477.0,257272477.0,gradinitagosen@yahoo.com,2018-2019,11269910,4,131,7
1,11256876,11192270,NaN,Anul şcolar 2018-2019,1061206943,NaN,ASOCIAȚIA PENTRU PROMOVAREA VALORILOR AUTENTIC...,ASOCIAȚIA VALORILOR BZ,Unitate de învățământ,Cu personalitate juridică,...,4,125300.0,762724000.0,238563970.0,secretariat@cnpaulescu.ro,2018-2019,11256876,0,0,0
2,11256865,11192168,NaN,Anul şcolar 2018-2019,1061206875,NaN,ASOCIAȚIA UMANITARĂ EDUSTAR BUZĂU,ASOCIATIA EDUSTAR,Unitate de învățământ,Cu personalitate juridică,...,14,120010.0,773826330.0,238725913.0,luminitamusat2006@yahoo.com,2018-2019,11256865,0,0,0


In [19]:
# Load csv school_levels_formations
levels_formations_Team4= pd.read_csv('school_levels_formations.csv')
levels_formations_Team4.head(3)

,id,idSchool,sfName,sfLevel,studyFormationType,educationType,fundingForm,educationForm,teachingMode,teachingLanguage,techingType,studentsNo
0,930621,11269910,B Îngerașii,Preșcolar,Grupa mică,Masă,Sponsorizare,Normal,Tradițional,Limba română,Normal,0
1,930620,11269910,A Ursuletii,Preșcolar,Grupa mică,Masă,Sponsorizare,Normal,Tradițional,Limba română,Normal,0
2,930623,11269910,A,Preșcolar,Grupa mare,Masă,Sponsorizare,Normal,Tradițional,Limba română,Normal,0


### Load data Tempo

In [29]:
csv_files = ['Elevi_cu_BAC.csv', 'Ateliere_scolare.csv', 'Laboratoare.csv', 'Echipamente_IT.csv', 
              'Terenuri_de_sport.csv', 'Imigranti.csv']

In [30]:
Tempo_all_Team4 = pd.DataFrame()
Tempo_all_Team4['Judet'] = ''
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    print(df.columns)
    if 'Judet' in df.columns:
        Tempo_all_Team4 = pd.merge(Tempo_all_Team4, df, on='Judet', how='outer')

Index(['Judet', 'Numar elevi'], dtype='object')
Index(['Judet', 'Atelire scolare'], dtype='object')
Index(['Judet', ' Valoare'], dtype='object')
Index(['Judet', 'Echipamente'], dtype='object')
Index(['Judet', 'Terenuri de sport'], dtype='object')
Index(['Judet', 'Imigranti'], dtype='object')


In [31]:
Tempo_all_Team4.rename(columns= {'Numar elevi': 'Elevi_cu_BAC', 'Atelire scolare':'Ateliere_scolare', ' Valoare':'Laboratoare',
                           'Terenuri de sport': 'Terenuri_sport' }, inplace=True)
Tempo_all_Team4.head()

,Judet,Elevi_cu_BAC,Ateliere_scolare,Laboratoare,Echipamente,Terenuri_sport,Imigranti
0,Bihor,2915,108,253,6137,73,300
1,Bistrita-Nasaud,1553,47,104,3814,21,177
2,Cluj,3653,112,368,14700,75,692
3,Maramures,2449,67,135,6815,37,326
4,Satu Mare,1460,74,139,3722,39,205


In [32]:
Tempo_all_Team4.to_csv('Tempo_all_Team4.csv', index=False)

### Connection details

In [33]:
database_name = "AID2023"
driver = '{ODBC Driver 18 for SQL Server}'
server_name = "aid-2023-srv.database.windows.net"

In [34]:
# Create the server url
server = 'tcp:{server_name},1433'.format(server_name = server_name)

#create the full connection string <- make sure to write it as it is bellow - check your credentials to be the right ones

connection_string = 'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};charset=utf8'.format(
    driver = driver,
    server=server,
    database=database_name,
    username=username,
    password=password
)                              

cnxn: pyodbc.Connection = pyodbc.connect(connection_string)
cursor: pyodbc.Cursor = cnxn.cursor() 

#execute a query with the cursor to check if the connection is maade
print("Connection is open") if cursor.execute("SELECT 1") else print(f"Connection is closed: {e}")

Connection is open


In [35]:
#obtain the parameters from the pyodbc connection string
params = urllib.parse.quote_plus(connection_string)
#create the engine
sqlalchemy_engine = db.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [36]:
# inserarea datelor intr-un tabel sql
details_Team4.to_sql("School_Details_Team4", sqlalchemy_engine, if_exists='replace', index=False) 
levels_formations_Team4.to_sql("Levels_Formations_Team4", sqlalchemy_engine, if_exists='replace', index=False) 
Tempo_all_Team4.to_sql("Tempo_all_Team4", sqlalchemy_engine, if_exists='replace', index=False) 

42